In [21]:
def remove_indet(frame, variable_name, keep_dontknow=False):
    ''' Removes entries from the dataframe for which the given variables has a negative, indeterminate value.'''
    frame = frame.query(f'{variable_name} != "-2"')
    frame = frame.query(f'{variable_name} != "-8"')
    frame = frame.query(f'{variable_name} != "-9"')
    if not keep_dontknow:
        frame = frame.query(f'{variable_name} != "-1"')
    return frame
    

In [1]:
import pandas

dataset = pandas.read_table("ICPSR_37692/DS0002/37692-0002-Data.tsv", low_memory=False)

In [ ]:
'''Select only self-identified transgender people.'''

trans_data = dataset.query('V1212 == 3')

In [22]:
'''
Select only data on non-straight people. This excludes V1213 answer 2, 
and also people who failed to indicate sexual orientation.

I include people who reported that they did not know their sexual orientation,
either via the -1 "Don't Know" error coding or the 5 "You don't know the answer."
This should allow for questioning people to be included as well.
'''

lgbq_data = dataset.query('V1213 != "2"') 
lgbq_data = remove_indet(lgb_data, "V1213", keep_dontknow=True)

In [24]:
'''
Select 'gender non-conforming' people. I define this to mean people whose assigned sex at birth does not match
their gender, regardless of transgender identification.
'''

gnc_data = dataset.query('V1211 != V1212')
remove_indet(gnc_data, 'V1211')
remove_indet(gnc_data, 'V1212')

,V0001,V0001B,V0002,V0003,V0004,V0005,V0006,V0007,V0008,V0009,...,RV0133,RV0134,RV0135,RV0136,RV0137,RV0138,RV0139,RV0140,RV0141,RV0142
10,999,11,999,1,1,1,2,1,1,999,...,999,999,999,999,999,999,999,999,999,999
451,999,452,999,1,1,1,2,1,1,999,...,999,999,999,999,999,999,999,999,999,999
478,999,479,999,1,1,1,2,1,1,999,...,999,999,999,999,999,999,999,999,999,999
652,999,653,999,1,1,1,2,1,1,999,...,999,999,999,999,999,999,999,999,999,999
719,999,720,999,1,1,1,2,1,1,999,...,999,999,999,999,999,999,999,999,999,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18812,999,18813,999,1,1,1,1,1,1,999,...,999,999,999,999,999,999,999,999,999,999
18968,999,18969,999,1,1,1,2,1,1,999,...,999,999,999,999,999,999,999,999,999,999
19450,999,19451,999,1,1,1,2,1,1,999,...,999,999,999,999,999,999,999,999,999,999
19596,999,19597,999,1,1,1,1,1,1,999,...,999,999,999,999,999,999,999,999,999,999


In [25]:
print(len(dataset))
print(len(trans_data))
print(len(lgbq_data))
print(len(gnc_data))

20064
29
1712
94
